In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import PreTrainedTokenizerFast, AutoModelForSequenceClassification, GPT2LMHeadModel


In [31]:
# 변수
# 그 외 추가 가능 인자 https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig

threshold=0.6 # float, 0~1
max_length = 50 # int, default=20
min_new_tokens = 2 # int
use_cache = True # bool, default=True
repetition_penalty = 5.0 # float, default=1.0
do_sample = True # bool, default=False
num_beams = 5 # int, default=1
temperature = 2.0 # float, default=1.0
top_k = 50 # int, default=50
top_p = 0.9 # float, default=1.0

In [32]:
cls_peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

gen_peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=True,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none")

In [33]:
Q_TKN = "<Q>"
A_TKN = "<A>"
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

### 분류 모델

In [34]:
# 저장된 모델 및 토크나이저 로드
cls_path = '/content/drive/MyDrive/Colab Notebooks/kogpt2-classification'
cls_model = AutoModelForSequenceClassification.from_pretrained(
      cls_path,
      num_labels=5,
      problem_type="multi_label_classification"
)
trained_cls_model = get_peft_model(cls_model, cls_peft_config)
trained_cls_tokenizer = PreTrainedTokenizerFast.from_pretrained(cls_path,
                          bos_token=BOS, eos_token=EOS, unk_token=UNK,
                          pad_token=PAD, mask_token=MASK)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
def predict_listener_empathy(input_text, model, tokenizer, threshold=threshold):
    # 모델을 평가 모드로 전환
    model.eval()

    # 입력 문장 토큰화
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # 모델에 입력을 전달하여 로짓(logits)을 얻음
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 로짓에 시그모이드 적용하여 확률로 변환
    probabilities = torch.sigmoid(logits)
    # 임계값을 기준으로 이진화
    predictions = (probabilities > threshold).int()

    # 레이블 디코딩
    label_classes = ['조언', '격려', '위로', '동조', '']
    num_classes = 5
    predicted_labels = [label_classes[i] for i in range(num_classes) if predictions[0][i] == 1]

    return predicted_labels


### 생성 모델

In [36]:
# 저장된 모델 및 토크나이저 로드
gen_path = '/content/drive/MyDrive/Colab Notebooks/kogpt2-chatbot'
gen_model = GPT2LMHeadModel.from_pretrained(gen_path)

trained_gen_model = get_peft_model(gen_model, gen_peft_config)
trained_gen_tokenizer = PreTrainedTokenizerFast.from_pretrained(gen_path,
                          bos_token=BOS, eos_token=EOS, unk_token=UNK,
                          pad_token=PAD, mask_token=MASK)

In [40]:
def predict_answer(predicted_labels, input_text, model, tokenizer,
                   max_length, min_new_tokens, use_cache,
                   repetition_penalty, do_sample, num_beams,
                   temperature, top_k, top_p):
    # 모델을 평가 모드로 전환
    model.eval()
    # 입력 문장 토큰화
    empathy = ' ,'.join(map(str, predicted_labels))
    inputs = Q_TKN + input_text + SENT + empathy + A_TKN
    input_ids = tokenizer.encode(tokenizer.bos_token + inputs + tokenizer.eos_token, return_tensors='pt')

    # 모델 추론
    outputs = model.generate(input_ids,
                             max_length=max_length, min_new_tokens=min_new_tokens, use_cache=use_cache,
                             repetition_penalty=repetition_penalty, do_sample=do_sample, num_beams=num_beams,
                             temperature=temperature, top_k=top_k, top_p=top_p, early_stopping=True)
    output_text = trained_gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
    output_text = output_text.split(A_TKN)[1]

    return output_text

## 추론

In [41]:
# 예제 입력 문장
input_text = "오늘 기분은 어때?..."

# 분류 결과 추론
# threshold 잘 설정해야
predicted_labels = predict_listener_empathy(input_text, trained_cls_model, trained_cls_tokenizer, threshold)
print(f"Predicted labels: {predicted_labels}")

Predicted labels: ['조언']


In [43]:
response = predict_answer(predicted_labels, input_text, trained_gen_model, trained_gen_tokenizer,
                   max_length, min_new_tokens, use_cache,
                   repetition_penalty, do_sample, num_beams,
                   temperature, top_k, top_p)
print(response)

 통찰력을 가진 자였으니까..
(중략 )
#책스타그램 #북스타그램 #독서스타그램 #독서 #
